In [1]:
import os
%pwd

'/mnt/d/vault/devhub/kidney_disease_classification/notebooks'

In [2]:
os.chdir("../")
%pwd

'/mnt/d/vault/devhub/kidney_disease_classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from src.kidney_disease_classification.constants import *
from src.kidney_disease_classification.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [6]:
import os
import tarfile
import gdown
from src.kidney_disease_classification import logger
from src.kidney_disease_classification.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self)->str:
        '''
        Fetch data from url
        '''

        try:
            dataset_url=self.config.source_URL
            zip_download_dir=self.config.local_data_file

            os.makedirs('artifacts/data_ingestion', exist_ok=True)
            logger.info(f'Downloading data from {dataset_url} into file {zip_download_dir}')

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, zip_download_dir)

            logger.info(f'Downloaded data from {dataset_url} into file {zip_download_dir}')
        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        '''
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        '''

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        file = tarfile.open(self.config.local_data_file)
        file.extractall(unzip_path)


In [8]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise 0

[2024-08-04 20:40:37,045: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-08-04 20:40:37,056: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-04 20:40:37,062: INFO: common: created directory at: artifacts]
[2024-08-04 20:40:37,066: INFO: common: created directory at: artifacts/data_ingestion]
[2024-08-04 20:40:37,070: INFO: 568940431: Downloading data from https://drive.google.com/file/d/1vS2MSaQ1Xyf4jkx1f3RNk5yQSimJ8wHR/view?usp=sharing into file artifacts/data_ingestion/data.tar.gz]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1vS2MSaQ1Xyf4jkx1f3RNk5yQSimJ8wHR
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1vS2MSaQ1Xyf4jkx1f3RNk5yQSimJ8wHR&confirm=t&uuid=963e98ee-7e2c-45ad-b5a7-c046227a664e
To: /mnt/d/vault/devhub/kidney_disease_classification/artifacts/data_ingestion/data.tar.gz
100%|██████████| 57.5M/57.5M [01:19<00:00, 723kB/s] 

[2024-08-04 20:42:01,778: INFO: 568940431: Downloaded data from https://drive.google.com/file/d/1vS2MSaQ1Xyf4jkx1f3RNk5yQSimJ8wHR/view?usp=sharing into file artifacts/data_ingestion/data.tar.gz]
